En este script entrenamos tanto los modelos tradicional (que servirán como baseline) como los modelos modernos basados en transformers para clasificación binaria de clickbaits.

In [1]:
import pandas as pd

df1 = pd.read_csv('dataset_clickbait_clasificacion_limpio.csv')
df2 = pd.read_csv('dataset_no_clickbait_clasificacion_limpio.csv')

# Unificación
df = pd.concat([df1, df2], ignore_index=True)

# Eliminación de duplicados
df.drop_duplicates(inplace=True)

Dividimos los datos en los conjuntos de entrenamiento (80%) y validación (20%).

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Asegurar que no hay NaN en la columna de texto
df = df.dropna(subset=['title'])

X = df['title']
y = df['is_clickbait']

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

Entrenamos los modelos tradicionales: SVM, Naive Bayes, Decision Tree y Random Forest.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

models = {
    "SVM": LinearSVC(),
    "Naive Bayes": MultinomialNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{name}\n")
    print(classification_report(y_test, y_pred))


SVM

              precision    recall  f1-score   support

           0       0.72      0.77      0.74        82
           1       0.73      0.67      0.70        76

    accuracy                           0.72       158
   macro avg       0.72      0.72      0.72       158
weighted avg       0.72      0.72      0.72       158


Naive Bayes

              precision    recall  f1-score   support

           0       0.71      0.73      0.72        82
           1       0.70      0.68      0.69        76

    accuracy                           0.71       158
   macro avg       0.71      0.71      0.71       158
weighted avg       0.71      0.71      0.71       158


Decision Tree

              precision    recall  f1-score   support

           0       0.67      0.62      0.65        82
           1       0.62      0.67      0.65        76

    accuracy                           0.65       158
   macro avg       0.65      0.65      0.65       158
weighted avg       0.65      0.65     

Entrenamos los modelos modernos.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
import evaluate
import torch

# Dataset HuggingFace
dataset = Dataset.from_pandas(df[['title', 'is_clickbait']].rename(columns={"title": "text", "is_clickbait": "label"}))
dataset = dataset.train_test_split(test_size=0.2)

# Preprocessing function
def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

# Evaluation metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Model list
model_list = [
    "dccuchile/bert-base-spanish-wwm-cased",
    "PlanTL-GOB-ES/roberta-base-bne",
    "bertin-project/bertin-roberta-base-spanish",
    "microsoft/mdeberta-v3-base",
    "cardiffnlp/twitter-xlm-roberta-base",
    "Twitter/twhin-bert-base",
    "dccuchile/distilbert-base-spanish-uncased",
    "CenIA/albert-base-spanish"
]

results = []

for model_name in model_list:
    print(f"\n🔍 Entrenando modelo: {model_name}\n")

    # Tokenizer y modelo
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Tokenización del dataset
    tokenized = dataset.map(preprocess, batched=True)

    # Entrenamiento
    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.replace('/', '_')}",
        eval_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        num_train_epochs=3,
        logging_dir=f"./logs_{model_name.replace('/', '_')}",
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluación
    eval_results = trainer.evaluate()
    results.append({
        "Modelo": model_name,
        "Accuracy": eval_results["eval_accuracy"]
    })


🔍 Entrenando modelo: dccuchile/bert-base-spanish-wwm-cased



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/628 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

/tmp/ipython-input-6-3351657420.py:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at CenIA/albert-base-spanish and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/628 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

/tmp/ipython-input-6-3351657420.py:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.597800,0.575582,0.715190
2,0.537200,0.528708,0.797468
3,0.431400,0.566902,0.816456


In [ ]:
# Comparativa de los resultados (ordenados de mayor a menor accuracy)
results_df = pd.DataFrame(results)
results_df.sort_values(by="Accuracy", ascending=False)

,Modelo,Accuracy
1,PlanTL-GOB-ES/roberta-base-bne,0.911392
0,dccuchile/bert-base-spanish-wwm-cased,0.905063
4,cardiffnlp/twitter-xlm-roberta-base,0.898734
3,microsoft/mdeberta-v3-base,0.892405
5,Twitter/twhin-bert-base,0.867089
6,dccuchile/distilbert-base-spanish-uncased,0.848101
7,CenIA/albert-base-spanish,0.816456
2,bertin-project/bertin-roberta-base-spanish,0.791139
